In [8]:
from dash import Dash, dash_table, dcc, html
from dash.dependencies import Input, Output
import pandas as pd
from flask import Flask
from Librerias_SAP import SAP_GUI, Funtions
import pandas as pd
import re
from collections import OrderedDict
from dash import dash_table as dt
from dash import dcc
from dash import html
from dash.dependencies import Input
from dash.dependencies import Output
import numpy as np
import dash_bootstrap_components as dbc



In [9]:
"""----------------------------Inciar session----------------------------------------------------"""
# Insert User name and password
Keys=pd.read_excel("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Claves\\Keys.xlsx")
user=Keys["User"][0]
password=Keys["Password"][0]
# Initialize session
session=SAP_GUI.SessionSAP(user,password)

In [10]:
# Disposiciones ZSD110
Name_ZSD110_FIRME="Pendiente_Firme"  # Revisar suele no guardar
Name_ZSD110_MERCADEO="Pendiente_Mercadeo"  # Revisar suele no guardar
Name_ZSD110_TOTAL="Pendiente_Total"  # Revisar suele no guardar


default_column_Canal01_Pendientes=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Pedido', 'Clase Pedi', 'Creado el', 'Pedido.1', 'Posición', 'Material',
       'Vlr.Neto P', 'Moneda', 'Cantidad P', 'Cantidad C', 'UM venta',
       'Vlr.Neto C', 'Status Glo', 'Status Tot', 'Status Ent', 'Status Ent.1',
       'Status Ent.2']  


default_column_Canal01_Entregado=['Can.distr.', 'Denomin.', 'Seccion', 'Estado', 'Solic.', 'Nombre',
       'Entrega', 'Posicion E', 'F.Creacion', 'Pedido', 'Clase Pedi',
       'F.Creacion.1', 'Pedido.1', 'Posicion P', 'Material', 'Moneda',
       'Cantidad E', 'UM venta', 'Valor Neto', 'StTotPick.', 'StatGlWM',
       'StTotMovMe', 'Stat.fact.', 'Status PT']


default_column_Pedidos=['OrgVt', 'CDis', 'BqEn', 'Solic.', 'PrimFeEntr', 'ÚltEntrega',
       'FePrefEnt.', 'ClVt', 'Valor neto', 'Mon.', 'Doc.venta', 'ST',
       'Solicitante', 'Nº pedido cliente', 'Descrip.breve', 'Nº de cliente 1',
       'Contador de pedidos', 'Dirección', 'Barrio', 'Población', 'Teléfono']

"""----------------------------------------------LEER ARCHIVOS--------------------------------"""
#Data Pendientes
Canal01_Pendiente=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_PEN.txt",skiprows=1,delimiter="\t")
Canal01_Pendiente=Funtions.Clean_Columns(Canal01_Pendiente)
Canal01_Pendiente=Funtions.default_column(default_column_Canal01_Pendientes,Canal01_Pendiente)

#Dta Entregados
Canal01_Entregado=pd.read_csv("C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Documentos 1\\SAP\\SAP GUI\\Canal01_ENT.txt",skiprows=1,delimiter="\t")
Canal01_Entregado=Funtions.Clean_Columns(Canal01_Entregado)
Canal01_Entregado=Funtions.default_column(default_column_Canal01_Entregado,Canal01_Entregado)


"""""-------------------------------------------Limpio los datos-----------------------------------"""
#Canal01_Pendiente['Vlr.Neto C'], Canal01_Pendiente['Cantidad P']=Funtions.Clean_Num_List(Canal01_Entregado['Vlr.Neto C'],Canal01_Entregado['Cantidad P'])
Canal01_Entregado['Valor Neto'], Canal01_Entregado['Cantidad E']=Funtions.Clean_Num_List(Canal01_Entregado['Valor Neto'],Canal01_Entregado['Cantidad E'])


In [11]:
"""-----------------Datos descriptivos de interes-----------------------------------"""
#Lineas Totales
Lines_Canal01_Pendiente=len(Canal01_Pendiente)
Lines_Canal01_Entregado=len(Canal01_Entregado)

In [12]:
"""------------------------Optener datos de Agenda_Exito y Cencosub------------------"""
#Agenda Exito y Cencosub
Agenda_Exito=["0085","0020","0146","0149","0050","0138","0045"]
Agenda_Cencosub=["93","122","127","95","60"]


"""-----------------------------Aplico filtro en Agenda exito----------------------------------"""
Filtro_Agenda_Exito=Funtions.Datos_Agenda(Canal01_Entregado['Pedido.1'],Agenda_Exito)
Filtro_Agenda_Cencosub=Funtions.Search_agenda(Canal01_Entregado["Pedido.1"],Agenda_Cencosub)


"""------------------------------Canal 01 Entrega con y sin agenda Exito-----------------------"""
Filtro_Canal01_Entregado_Exito=(Canal01_Entregado['Pedido.1'].isin(Filtro_Agenda_Exito))   #Excluyo con ~
Canal01_Entregado_Agenda_Exito=Canal01_Entregado.loc[Filtro_Canal01_Entregado_Exito].reset_index(drop=True)
Canal01_Entregado_Sin_Agenda_Exito=Canal01_Entregado.loc[~Filtro_Canal01_Entregado_Exito].reset_index(drop=True)


"""------------------------------Canal 01 Entrega con y sin agenda Cencosub-----------------------"""
Filtro_Canal01_Entregado_Cencosub=(Canal01_Entregado['Pedido.1'].isin(Filtro_Agenda_Cencosub))   #Excluyo con ~
Canal01_Entregado_Agenda_Cencosub=Canal01_Entregado.loc[Filtro_Canal01_Entregado_Cencosub].reset_index(drop=True)
Canal01_Entregado_Sin_Agenda_Cencosub=Canal01_Entregado.loc[~Filtro_Canal01_Entregado_Cencosub].reset_index(drop=True)


In [13]:
#Creo las tablas dinamicas de cliente Exito y Cencosub

Table_dinamica_Exito_entrega=Canal01_Entregado_Agenda_Exito.pivot_table(index=['Pedido.1','Clase Pedi'],aggfunc={'Material':'count','Cantidad E':sum,'Valor Neto':sum}).reset_index()
Lineas_table_dinamica_Exito_entrega=sum(Table_dinamica_Exito_entrega["Material"])


Table_dinamica_Cencosub_entrega=Canal01_Entregado_Agenda_Cencosub.pivot_table(index=['Pedido.1','Clase Pedi'],aggfunc={'Material':'count','Cantidad E':sum,'Valor Neto':sum}).reset_index()
Lineas_table_dinamica_Cencosub_entrega=sum(Table_dinamica_Cencosub_entrega["Material"])

In [31]:
pedidos_Exito_Cencosub=pd.concat([Table_dinamica_Cencosub_entrega['Pedido.1'],Table_dinamica_Exito_entrega['Pedido.1']])

#Busco en la tansaccion ZSD037 los pedidos de las tablas dinamicas tanto de Exito y Cencosub

Transsaccion_ZSD037="zsd037"
SAP_GUI.Search_Pedidos_ZSD037(Transsaccion_ZSD037,pedidos_Exito_Cencosub,session)

In [33]:
Name_ZSD037="Pedidos pendientes Exito y Cencosub"
Ruta_ZSD037_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"
SAP_GUI.Export_TXT2(Name_ZSD037,session,Ruta_ZSD037_Exito_Cencosub)

In [130]:
# ZSD110 FIRME
url_ZSD037_Pedidos_Exito_Cencosub="C:\\Users\\prac.ingindustrial2\\OneDrive - Prebel S.A\\Escritorio\\SAP\\Archivos_CSV\\"+str(Name_ZSD037)+".txt"

Data_ZSD110_Pedidos_Exito_Cencosub=pd.read_csv(url_ZSD037_Pedidos_Exito_Cencosub,skiprows=5,delimiter="\t")
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.Clean_Columns(Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Funtions.default_column(default_column_Pedidos,Data_ZSD110_Pedidos_Exito_Cencosub)
Data_ZSD110_Pedidos_Exito_Cencosub=Data_ZSD110_Pedidos_Exito_Cencosub[Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente'].notnull()]



In [131]:
Data_ZSD110_Pedidos_Exito_Cencosub.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4940 entries, 0 to 4939
Data columns (total 21 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   OrgVt                4940 non-null   float64
 1   CDis                 4940 non-null   float64
 2   BqEn                 0 non-null      float64
 3   Solic.               4940 non-null   float64
 4   PrimFeEntr           4940 non-null   object 
 5   ÚltEntrega           4940 non-null   object 
 6   FePrefEnt.           4940 non-null   object 
 7   ClVt                 4940 non-null   object 
 8   Valor neto           4940 non-null   object 
 9   Mon.                 4940 non-null   object 
 10  Doc.venta            4940 non-null   float64
 11  ST                   4940 non-null   object 
 12  Solicitante          4940 non-null   object 
 13  Nº pedido cliente    4940 non-null   object 
 14  Descrip.breve        4940 non-null   object 
 15  Nº de cliente 1      4417 non-null   o

In [132]:


#Limpiar colummna a numeros
def Clean_int_to_str(*args):
    lista=[]
    for arg in args:
        args=arg.astype(int).astype(str)
        lista.append(args)
    if len(lista)==1:
        return lista[0]
    else:
        return(tuple(lista))
try:
    Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente']=Clean_int_to_str(Data_ZSD110_Pedidos_Exito_Cencosub['Nº pedido cliente'])   #OrgVt   #CDis  #4940
except:
    None  #No ese aplica metódo

In [136]:
#Dataframe de interes
Data_ZSD110_Pedidos_Pendientes_Fecha=Data_ZSD110_Pedidos_Exito_Cencosub[['Nº pedido cliente','PrimFeEntr','ÚltEntrega']].drop_duplicates()


,Nº pedido cliente,PrimFeEntr,ÚltEntrega
0,0020631946,01.06.2023,01.06.2023
87,0050432797,03.06.2023,03.06.2023
97,0050432798,03.06.2023,03.06.2023
107,0050432800,03.06.2023,03.06.2023
117,0050432811,03.06.2023,03.06.2023
407,0050432812,03.06.2023,03.06.2023
425,0050432813,03.06.2023,03.06.2023
606,0050432814,03.06.2023,03.06.2023
616,0050432815,03.06.2023,03.06.2023
724,0050432822,03.06.2023,03.06.2023


In [137]:
#---------------------------------Tabla dinamica Exito---------------------------------------------
Tabla_Agenda_Exito=pd.merge(Data_ZSD110_Pedidos_Pendientes_Fecha,Table_dinamica_Exito_entrega,how="right",left_on="Nº pedido cliente",right_on="Pedido.1")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedi', 'Cantidad E', 'Material', 'Valor Neto']]

Agenda_name_Exito={"0085":"FUNZA","0020":"VEGAS","0146":"BARRANQUILLA","0149":"BUCARAMANGA","0050":"CALI","0138":"PEREIRA","0045":"SURTIMAYORISTAS"}

Tabla_Agenda_Exito["Plataforma"]=Tabla_Agenda_Exito["Nº pedido cliente"].apply(lambda x: Agenda_name_Exito[re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)[0]] if re.findall("("+"|".join(list(Agenda_name_Exito.keys()))+")",x)!=[] else x)


In [140]:
#Dataframe de interes

Tabla_Agenda_Cencusub=pd.merge(Data_ZSD110_Pedidos_Pendientes_Fecha,Table_dinamica_Cencosub_entrega,how="right",left_on="Nº pedido cliente",right_on="Pedido.1")[['Nº pedido cliente', 'PrimFeEntr', 'ÚltEntrega',
       'Clase Pedi', 'Cantidad E', 'Material', 'Valor Neto']]

Agenda_name_Cencosub={"93-":"MEDELLIN","122-":"BARRANQUILLA","127-":"BUCARAMANGA","95-":"CALI","60-":"BOGOTA"}


Tabla_Agenda_Cencusub["Plataforma"]=Tabla_Agenda_Cencusub["Nº pedido cliente"].apply(lambda x: Agenda_name_Cencosub[re.findall("\d*-",x)[0]] if re.findall("\d*-",x)[0] in list(Agenda_name_Cencosub.keys()) else None)


In [141]:
Tabla_Agenda_Cencusub

,Nº pedido cliente,PrimFeEntr,ÚltEntrega,Clase Pedi,Cantidad E,Material,Valor Neto,Plataforma
0,122-439976,08.06.2023,08.06.2023,ZVMI,723.0,334,11598587.0,BARRANQUILLA
1,127-443841,08.06.2023,08.06.2023,ZVMI,1820.0,544,28247168.0,BUCARAMANGA
2,60-1158908,02.06.2023,02.06.2023,ZEDI,118.0,25,5119346.0,BOGOTA
3,60-1158916,02.06.2023,04.06.2023,ZEDI,12.0,1,378348.0,BOGOTA
4,60-1159026,05.06.2023,05.06.2023,ZVMI,6098.0,2524,93697730.0,BOGOTA
5,93-485553,08.06.2023,08.06.2023,ZVMI,1008.0,476,15120226.0,MEDELLIN
6,95-493513,08.06.2023,08.06.2023,ZVMI,1123.0,389,14674390.0,CALI


In [117]:
y

2